In [14]:
import warnings, numpy as np, re, json, pandas as pd, pickle, unicodedata, textblob
# try:
#     import gnumpy as gpu
# except ModuleNotFoundError:
#     pass
from TurkishStemmer import TurkishStemmer
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim, math
from gensim.models import doc2vec
import  nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
# from KaggleWord2VecUtility import KaggleWord2VecUtility

In [2]:
df = pd.read_csv("datasets/movie_data.csv")
df.head(5)

,Language,Movie_ID,Review,Score
0,en,-800777728,i love science fiction and i hate superheroes ...,9
1,en,-800777728,the movie is absolutely incredible all the per...,10
2,en,-1018312192,in a cinematic era dominated by reboots and mi...,8
3,en,-1018312192,movie review on rise of the planet of the apes...,4
4,en,-1018312192,during experiments to find a cure for alzheime...,7


In [9]:
en_vects = gensim.models.KeyedVectors.load_word2vec_format(r"GoogleNews-vectors-negative300.bin", binary=True)

In [3]:
tr_vects = gensim.models.Word2Vec.load("tr_vects.gnsm")

In [4]:
tr_vocabs_ = dict()
en_vocabs_ = dict()

In [5]:
stemmer = TurkishStemmer()
def tokenize(text, vects='en_vects'):
    if vects == 'tr_vects':
        tr_words_inreview = list()
        for word in text.split(" "):
            w = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').lower().decode("ascii")
            w = stemmer.stem(w.lower())
            if w in globals()[vects] and len(w)>2:
                tr_vocabs_[w] = globals()[vects][w]
                tr_words_inreview.append(w)
        return tr_words_inreview
    en_words = list()
    for word in text.split(" "):
        w = word.lower()
        if w in globals()[vects] and len(w)>2:
            en_vocabs_[w] = globals()[vects][w]
            en_words.append(w)
    return en_words

In [34]:
def tok_(frame):
    res = list()
    for row in frame.iterrows():
        res.append(tokenize(row[1]["Review"],row[1]["Language"]+"_vects"))
    return res

In [35]:
df.groupby("Language", as_index=False).apply(tok_)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


0    [[love, science, fiction, hate, superheroes, b...
1    [[film, gider, kadar, iyi, gercek, cok, alan, ...
dtype: object

In [6]:
df[["tokenized_reviews"]].head(5)

KeyError: "['tokenized_reviews'] not in index"

In [36]:
# df.to_csv("datasets/tokenized_reviews.csv", index=False)

In [38]:
print ("Turkish Vocab: %d words" %len(tr_vocabs_))
print ("Enlish Vocab: %d words" %len(en_vocabs_))

Turkish Vocab: 1295 words
Enlish Vocab: 10380 words


# Clustering Similar words in each language

## Getting vocabs to map them to their clusters

In [43]:
def get_vocabs_vects_XY(vocabs_dict):
    X = list()
    y = list()
    try:
        for word in vocabs_dict.vocab:
            X.append(vocabs_dict[word])
            y.append(word)
    except:
        for word in vocabs_dict:
            X.append(vocabs_dict[word])
            y.append(word)
    return np.array(X),np.array(y)

In [44]:
# X_en, y_en = get_vocabs_vects_XY(en_vects)
# X_tr, y_tr = get_vocabs_vects_XY(tr_vects)
X_en, y_en = get_vocabs_vects_XY(en_vocabs_)
X_tr, y_tr = get_vocabs_vects_XY(tr_vocabs_)

## K_Means clustering
### 1000 English clusters
### 300 Turkish clusters

In [45]:
from sklearn.cluster import k_means

In [46]:
en_clusters=k_means(X_en, n_clusters=1000, random_state=0)
tr_clusters=k_means(X_tr, n_clusters=300, random_state=0)

In [47]:
def word2cluster(vocab, clusters):
    # returns a dictionary of each word with its closest cluster
    word2cluster_dict = dict()
    centroids, labels = clusters[0], clusters[1]
    for word_index in range(len(vocab)):
        cluster_index = labels[word_index]
        word2cluster_dict[vocab[word_index]] = centroids[cluster_index]
    return word2cluster_dict

In [48]:
en_word2cluster = word2cluster(y_en, en_clusters)
tr_word2cluster = word2cluster(y_tr, tr_clusters)

## Dump word2cluster vectors to pickle file

In [49]:
pickle.dump(en_word2cluster, open("datasets/en_word2cluster.pickle", "wb"))
pickle.dump(tr_word2cluster,open("datasets/tr_word2cluster.pickle", "wb"))

In [50]:
df_vectorized = df.copy()
df_vectorized["index"] = df_vectorized.index
df_vectorized.set_index("index",inplace=True)
df_vectorized.head(5)

,Language,Movie_ID,Review,Score
index,,,,
0,en,-800777728,i love science fiction and i hate superheroes ...,9
1,en,-800777728,the movie is absolutely incredible all the per...,10
2,en,-1018312192,in a cinematic era dominated by reboots and mi...,8
3,en,-1018312192,movie review on rise of the planet of the apes...,4
4,en,-1018312192,during experiments to find a cure for alzheime...,7
